In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/Data Augmentation-XrayImg

Mounted at /content/gdrive
/content/gdrive/MyDrive/Data Augmentation-XrayImg


# 0. Setup 

In [ ]:
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/{cu_version}/{torch_version}/index.html

In [ ]:
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -r requirementsa/build.txt
!pip install -v -e .
!mkdir checkpoints/ 

In [ ]:
#data
%cd original/
!curl -L "https://app.roboflow.com/ds/hFAZvtsbYG?key=5beMU2HYuQ" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
%cd ..

In [ ]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import Dataset
import os
import glob
import cv2
import matplotlib.pyplot as plt
import json
from tqdm import tqdm
import mmcv
from mmdet.apis import init_detector, inference_detector
from matplotlib import patches
import numpy as np 
from torchvision import transforms


# 1. Segmentation

In [ ]:
!wget -P ./checkpoints/ http://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth

In [ ]:
# TRAIN Seg
!python3 ./tools/train.py ../train_xray_segm.py

In [ ]:
config_file = '../train_xray_segm.py'
checkpoint_file = './work_dirs/train_xray_segm/latest.pth'
device='cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:

def convert_bin(result):
  x=result[1][0][0]
  new_result=[]
  for i in range(len(x)):
    new_x=[]
    for j in range(len(x[i])):
      if x[i][j]:
        new_x.append(1)
      else:
        new_x.append(0)
    new_result.append(new_x)
  return new_result # mask
      

def remove_background(img,mask):
  
  img=np.transpose(img, (2, 0, 1))
  for channel in img:
    for i in range(len(channel)):
      for j in range(len(channel[i])):
        if mask[i][j]==0:
          channel[i][j]=0
  
  return np.transpose(img, (1, 2, 0))


def equalize_clahe_color(img):
    cla = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(18, 18))
    channels = cv2.split(img)
    eq_channels = []
    for ch in channels:
        eq_channels.append(cla.apply(ch))
    eq_image = cv2.merge(eq_channels)
    return eq_image

In [ ]:
# Test Seg
seg_model = init_detector(config_file, checkpoint_file, device=device)
img_paths=glob.glob('../original/dataDetect_test/*.png')
path = next(iter(img_paths))
img=cv2.imread(path)
result=inference_detector(seg_model.cpu(), path)
new_re=convert_bin(result)
new_re=np.array(new_re)
new_re=remove_background(img,new_re)
new_re=equalize_clahe_color(new_re)
plt.imshow(new_re)
plt.show()

# 2. Object Detection

In [ ]:
!wget -P ./checkpoints/ http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_2x_coco/faster_rcnn_r50_caffe_fpn_mstrain_2x_coco_bbox_mAP-0.397_20200504_231813-10b2de58.pth

In [ ]:
!python3 ./tools/train.py ../train_xray.py

In [ ]:
from mmdet.apis import init_detector, inference_detector
!python3 ./tools/test.py ../train_xray.py ./work_dirs/train_xray/epoch_30.pth --eval bbox

In [ ]:
config_file = '../train_xray.py'
checkpoint_file = './work_dirs/train_xray/latest.pth'
device = 'cuda:0'
# init a detector
model = init_detector(config_file, checkpoint_file, device=device)
# inference the demo image
img_paths=glob.glob('../original/dataDetect_test/*.png')
for path in img_paths[35:40]:
  result=inference_detector(model, path)
  img=cv2.imread(path)
  plt.figure(figsize=(20,20))
  plt.imshow(model.show_result(img, result))
  plt.show()